In [1]:
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model
labels_df = pd.read_csv('product_attributes_updated.csv', encoding='utf-8', 
                    dialect='excel', lineterminator='\n')

Using TensorFlow backend.


In [2]:
labels_df = labels_df.drop(labels_df[labels_df.Style1=='Southwestern'].index)
labels_df = labels_df.drop(labels_df[labels_df.Style3=='Southwestern'].index)
labels_df = labels_df.drop(labels_df[labels_df.Style1=='French Country'].index)
labels_df = labels_df.drop(labels_df[labels_df.Style2=='French Country'].index)

In [3]:
 def image2np_array( filename ) :
    img = Image.open( filename )
    img.load()
    data = np.asarray( img, dtype='uint8' )
    return data

Random_Product=labels_df['ProductID'].sample().iloc[0]

try:
    img = image2np_array('..\\ml_150x150\\P994212.jpg').reshape(1,150,150,3)
except (IOError, ValueError) as e:
    print('{0}'.format(e))
try:
    img_m = image2np_array('..\\ml_150x150_m\\P994212.jpg').reshape(1,150,150,1)
except (IOError, ValueError) as e:
    print('{0}'.format(e))
print ('true labels')
labels_df[labels_df['ProductID']==Random_Product]

true labels


,ProductID,EID,PID,VID,Department,Category,Type,Product Title,Finish1,Treatment,Hanging Method,Shade Color,Shade Shape,Style1,Style2,Style3,Tier
5710,P1017024,NaN,P1017024,CL990,Lamps,Table Lamps,Table Lamps,Robert Abbey Harvey Table Lamp,Green Tones,Polished,NaN,White Tones,Drum,Contemporary / Modern,NaN,NaN,NaN


In [4]:
def delete_rare_records(treshold,column_name):
    global labels_df
    e = labels_df[column_name].value_counts()
    return labels_df[labels_df[column_name].isin(e[e>treshold].index)]

In [5]:
style_keywords=['Bronze','Brown','Satin','Chrome','Black','Brass','Gold','Polished','White','Silver','Copper','Nickel','Sconces','Bathroom','Outdoor','Pendant','Mini-Pendant','Chandeliers','Table','Flushmount','Semi-Flushmount','Multi-Light','Post','Swing','Mini-Chandeliers','Hanging','Island','Floor','Crystal','Wall','Sconce','Lamp']
style_dummy=[0]*(len(style_keywords))
for x in style_keywords:
    if x in labels_df[labels_df['ProductID']==Random_Product]['Product Title'].values[0].split(): 
        style_dummy[style_keywords.index(x)]=1

style_model = load_model('style_classifier.h5')
style_prediction=style_model.predict([img,np.array([style_dummy])], batch_size=1)

style_columns=pd.get_dummies(labels_df['Style1']).columns
style_prediction_df = pd.DataFrame(data=style_prediction, columns=style_columns)
style_prediction_df.loc[:, (style_prediction_df > 0.3).any(axis=0)]

,Art Glass,Contemporary / Modern
0,0.431403,0.910101


In [6]:
treatment_keywords=['Bronze','Brown','Satin','Polished','Brushed','Antique','Oil','Matte','Patina','Textured','Wall','Sconce','Lamp','Nickel']
treatment_dummy=[0]*(len(treatment_keywords))
for x in treatment_keywords:
    if x in labels_df[labels_df['ProductID']==Random_Product]['Product Title'].values[0].split(): 
        treatment_dummy[treatment_keywords.index(x)]=1

treatment_model = load_model('Treatment_classifier.h5')
treatment_prediction=treatment_model.predict([img,np.array([treatment_dummy])], batch_size=1)

treatment_columns=pd.get_dummies(delete_rare_records(treshold=200,column_name='Treatment')['Treatment']).columns
treatment_prediction_df = pd.DataFrame(data=treatment_prediction, columns=treatment_columns)
treatment_prediction_df.loc[:, (treatment_prediction_df > 0.3).any(axis=0)]


""
0


In [7]:
type_keywords=['Sconces','Bathroom','Outdoor','Pendant','Mini-Pendant','Chandeliers','Table','Flushmount','Semi-Flushmount','Multi-Light','Post','Swing','Mini-Chandeliers','Hanging','Island','Floor','Crystal','Wall','Sconce','Lamp']
type_dummy=[0]*(len(type_keywords))
for x in type_keywords:
    if x in labels_df[labels_df['ProductID']==Random_Product]['Product Title'].values[0].split(): 
        type_dummy[type_keywords.index(x)]=1

type_model = load_model('type_classifier.h5')
type_prediction=type_model.predict([img,np.array([type_dummy])], batch_size=1)

type_columns=pd.get_dummies(delete_rare_records(treshold=300,column_name='Type')['Type']).columns
type_prediction_df = pd.DataFrame(data=type_prediction, columns=type_columns)
type_prediction_df.loc[:, (type_prediction_df > 0.3).any(axis=0)]

,Table Lamps
0,0.851125


In [8]:
finish_keywords=['Bronze','Brown','Satin','Chrome','Black','Brass','Gold','Polished','White','Silver','Copper','Wall','Sconce','Lamp','Nickel']
finish_dummy=[0]*(len(finish_keywords))
for x in finish_keywords:
    if x in labels_df[labels_df['ProductID']==Random_Product]['Product Title'].values[0].split(): 
        finish_dummy[finish_keywords.index(x)]=1

finish_model = load_model('Finish1_classifier.h5')
finish_prediction=finish_model.predict([img,np.array([finish_dummy])], batch_size=1)

finish_columns=pd.get_dummies(delete_rare_records(treshold=300,column_name='Finish1')['Finish1']).columns
finish_prediction_df = pd.DataFrame(data=finish_prediction, columns=finish_columns)
finish_prediction_df.loc[:, (finish_prediction_df > 0.3).any(axis=0)]

,Bronze / Brown Tones
0,0.343767


In [9]:
shade_color_keywords=['White','Clear','Beige','Cream','None','Brown','Amber','Alabaster','Art','Multi-Color','Bronze','Red','Black','Blue','Satin','Brass','Wall','Sconce','Lamp','Nickel']
shade_color_dummy=[0]*(len(shade_color_keywords))
for x in shade_color_keywords:
    if x in labels_df[labels_df['ProductID']==Random_Product]['Product Title'].values[0].split(): 
        shade_color_dummy[shade_color_keywords.index(x)]=1

        
shade_color_model = load_model('Shade_Color_classifier.h5')
shade_color_prediction=shade_color_model.predict([img,np.array([shade_color_dummy])], batch_size=1)

shade_color_columns=pd.get_dummies(delete_rare_records(treshold=300,column_name='Shade Color')['Shade Color']).columns
shade_color_prediction_df = pd.DataFrame(data=shade_color_prediction, columns=shade_color_columns)
shade_color_prediction_df.loc[:, (shade_color_prediction_df > 0.3).any(axis=0)]

,White Tones
0,0.427797


In [10]:
shade_shape_keywords=['Bowl','Dome','Cylindrical','Bell','Drum','Rectangle','Square','Conical','Oval','Empire','Coolie','Globe','Wall','Sconce','Lamp']
shade_shape_dummy=[0]*(len(shade_shape_keywords))
for x in shade_shape_keywords:
    if x in labels_df[labels_df['ProductID']==Random_Product]['Product Title'].values[0].split(): 
        shade_shape_dummy[shade_shape_keywords.index(x)]=1


shade_shape_model = load_model('Shade_Shape_classifier.h5')
shade_shape_prediction=shade_shape_model.predict([img_m,np.array([shade_shape_dummy])], batch_size=1)

shade_shape_columns=pd.get_dummies(delete_rare_records(treshold=200,column_name='Shade Shape')['Shade Shape']).columns
shade_shape_prediction_df = pd.DataFrame(data=shade_shape_prediction, columns=shade_shape_columns)
shade_shape_prediction_df.loc[:, (shade_shape_prediction_df > 0.3).any(axis=0)]

,Bowl / Dome
0,0.640923


In [14]:
hanging_metod_keywords=['Chain','Stem','Cord','Cable','Combination','Wall','Sconce','Lamp']
hanging_metod_dummy=[0]*(len(hanging_metod_keywords))
for x in hanging_metod_keywords:
    if x in labels_df[labels_df['ProductID']==Random_Product]['Product Title'].values[0].split(): 
        hanging_metod_dummy[hanging_metod_keywords.index(x)]=1


hanging_metod_model = load_model('Hanging_metod_classifier.h5')
hanging_metod_prediction=hanging_metod_model.predict([img_m,np.array([hanging_metod_dummy])], batch_size=1)

hanging_metod_columns=pd.get_dummies(labels_df['Hanging Method']).columns
hanging_metod_prediction_df = pd.DataFrame(data=hanging_metod_prediction, columns=hanging_metod_columns)
hanging_metod_prediction_df.loc[:, (hanging_metod_prediction_df > 0.3).any(axis=0)]

,Cord Hung
0,0.99962


In [15]:
tier_keywords=['Bronze','Brown','Satin','Chrome','Black','Brass','Gold','Polished','White','Silver','Copper','Wall','Sconce','Lamp','Nickel']
tier_dummy=[0]*(len(tier_keywords))
for x in tier_keywords:
    if x in labels_df[labels_df['ProductID']==Random_Product]['Product Title'].values[0].split(): 
        tier_dummy[tier_keywords.index(x)]=1

tier_model = load_model('Tier_classifier.h5')
tier_prediction=tier_model.predict(img_m, batch_size=1)

tier_columns=pd.get_dummies(labels_df['Tier']).columns
tier_prediction_df = pd.DataFrame(data=tier_prediction, columns=tier_columns)
tier_prediction_df.loc[:, (tier_prediction_df > 0.3).any(axis=0)]

,1.0
0,1.0
